# Library

In [ ]:
import numpy as np
import os, sys, glob
from astropy.io import fits
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
import matplotlib.pyplot as plt
import time
from astropy.io import ascii
# from imsng import tool_tbd
from astroquery.imcce import Skybot
from astropy.time import Time
from astropy.nddata import Cutout2D
from astropy.visualization import SqrtStretch, LinearStretch, LogStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization import ZScaleInterval, MinMaxInterval
from matplotlib.patches import Circle, PathPatch


: 

In [ ]:
#	plot setting
import matplotlib as mpl
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"

mpl.rcParams["axes.titlesize"] = 14
mpl.rcParams["axes.labelsize"] = 20
plt.rcParams['savefig.dpi'] = 500
plt.rc('font', family='serif')

: 

# Function

In [ ]:
#============================================================
#	FUNCTION
#============================================================
def trim(inim, position, size, outim='trim.fits'):
	# Load the image and the WCS
	hdu = fits.open(inim)[0]
	wcs = WCS(hdu.header)
	# Make the cutout, including the WCS
	cutout = Cutout2D(hdu.data, position=position, size=size, wcs=wcs)
	# Put the cutout image in the FITS HDU
	hdu.data = cutout.data
	# Update the FITS header with the cutout WCS
	hdu.header.update(cutout.wcs.to_header())
	# Write the cutout to a new FITS file
	hdu.writeto(outim, overwrite=True)
#------------------------------------------------------------
def hotpants(inim, refim, outim='hd.fits', convim='hc.fits'):
	'''
	inim : Science image
	refim : Reference image
	'''
	# path = os.path.dirname(inim)
	# image = os.path.basename(inim)
	# outim = f'hd{image}'
	# convim = f'hc{image}'
	# com = f'hotpants -c t -n i -iu 60000 -tu 6000000000 -tl -100000 -v 0 -inim {inim} -tmplim {refim} -outim {outim} -oci {convim}'
	com = f'hotpants -c t -n i -iu 6000000000 -il -100000 -tu 6000000000 -tl -100000 -v 0 -inim {inim} -tmplim {refim} -outim {outim} -oci {convim}'
	print(com)
	os.system(com)
#------------------------------------------------------------
def wcsremap(inim, refim, outim, path_com='/data3/wcsremap/wcsremap-1.0.1/wcsremap'):
	import os
	com = f'{path_com} -template {refim} -source {inim} -outim {outim}'
	print(com)
	os.system(com)
	# return outim
#------------------------------------------------------------
def invert_image(inim, outim):
	data, hdr = fits.getdata(inim, header=True)
	# w = WCS(inim)
	invdata = data*(-1)
	fits.writeto(outim, invdata, header=hdr, overwrite=True)
#------------------------------------------------------------
def sexcom(inim, conf_sex, conf_param, conf_conv, conf_nnw,):
	outcat = inim.replace('fits', 'cat')
	sexcom = 'sex {} -c {} -CATALOG_NAME {} -PARAMETERS_NAME {} -FILTER_NAME {} -STARNNW_NAME {}'.format(inim, conf_sex, outcat, conf_param, conf_conv, conf_nnw)
	# print(sexcom)
	# os.system(sexcom)
	return sexcom
#------------------------------------------------------------
def findloc(inim):
	path_table = "/home/paek/qsopy/tables"
	loctbl = ascii.read(f'{path_table}/obs.location.smnet.dat')
	#	code == 500 : default
	code = 500
	for i, obs in enumerate(loctbl['obs']):
		if obs in inim:
			code = loctbl['Code'][i].item()
			break
	return code
#------------------------------------------------------------
def generate_snapshot(trtbl, i, cutsize=2.0):
	#	Images
	n = trtbl['NUMBER'][i].item()
	inim, hcim, hdim = trtbl['inim'][i], trtbl['hcim'][i], trtbl['hdim'][i]
	#	Poistion of transient candidate
	tra = trtbl['ALPHA_J2000'][i].item()
	tdec = trtbl['DELTA_J2000'][i].item()
	position = SkyCoord(tra, tdec, frame='icrs', unit='deg')
	#	Seeing
	seeing = trtbl.meta['SEEING']
	# peeing = trtbl['peeing'][i].item()
	size = u.Quantity((cutsize, cutsize), u.arcmin)

	for image, kind in zip([inim, hcim, hdim], ['new', 'ref', 'sub']):
		hdu = fits.open(image)[0]
		wcs = WCS(hdu.header)
		peeing = seeing/0.4
		# peeing = hdu.header['PEEING']
		# Make the cutout, including the WCS
		cutout = Cutout2D(hdu.data, position=position, size=size, wcs=wcs)
		data = 	cutout.data
		# Put the cutout image in the FITS HDU
		hdu.data = cutout.data
		# Update the FITS header with the cutout WCS
		hdu.header.update(cutout.wcs.to_header())
		# Write the cutout to a new FITS file
		outim = f'{os.path.splitext(hdim)[0]}.{n:0>6}.{kind}{os.path.splitext(hdim)[1]}'
		outpng = f'{os.path.splitext(hdim)[0]}.{n:0>6}.{kind}.png'
		#	Save postage stamp *.png & *.fits
		hdu.writeto(outim, overwrite=True)
		plot_snapshot(data, wcs, peeing, outpng, save=True)
#------------------------------------------------------------
def plot_snapshot(data, wcs, peeing, outpng, save=True):
	plt.close('all')
	plt.rc('font', family='serif')
	fig = plt.figure(figsize=(1, 1))
	fig.set_size_inches(1. * data.shape[0] / data.shape[1], 1, forward = False)
	x = 720 / fig.dpi
	y = 720 / fig.dpi
	fig.set_figwidth(x)
	fig.set_figheight(y)
	#	No axes
	# ax = plt.subplot(projection=wcs)
	ax = plt.Axes(fig, [0., 0., 1., 1.])
	ax.set_axis_off()
	fig.add_axes(ax)

	from astropy.visualization.stretch import LinearStretch
	#	Sci
	data[np.isnan(data)] = 0.0
	transform = LinearStretch()+ZScaleInterval()
	bdata = transform(data)
	# pylab.subplot(131)
	ax.imshow(bdata, cmap="gray", origin="lower")

	#	Circle
	circle = Circle(
		(data.shape[0]/2., data.shape[1]/2.),
		2*peeing,
		edgecolor='yellow',
		lw=3,
		facecolor=None,
		fill=False
	)

	ax.add_patch(circle)

	#	RA, Dec direction
	ra0, dec0 = wcs.all_pix2world(0, 0, 1)
	ra1, dec1 = wcs.all_pix2world(data.shape[0], data.shape[1], 1)
	if ra0>ra1:
		pass
	elif ra0<ra1:
		ax.invert_xaxis()
	if dec0>dec1:
		ax.invert_yaxis()
	elif dec0<dec1:
		pass
	#	Save or not?
	if save:
		plt.savefig(outpng, dpi=100,)
	else:
		pass
#------------------------------------------------------------
def rename_convention(inim):
	hdr = fits.getheader(inim)
	"""
	Special name for splited KMTNet images
	"""
	observat = hdr['OBSERV0']
	obs = f'KMTNet_{observat}'
	obj = hdr['OBJECT']
	exptime = hdr['EXPTIME']
	filte = hdr['FILTER']
	# dateobs = hdr['DATE-OBS'].replace('-', '').replace(':', '').replace('T', '-')
	dateobs = hdr['DATE'].replace('-', '').replace(':', '').replace('T', '-')
	#       New name
	newim = f"{os.path.dirname(inim)}/Calib.{obs}.{obj}.{dateobs}.{filte}.{exptime:g}.stack.fits"
	return newim


: 

# Set the images

In [ ]:
path_science_image = '../grb230307a'
science_images = sorted(glob.glob(f"{path_science_image}/TOO*.fits"))
reference_image = '../grb230307a/REF_0503.061-76.R.20191207.SAAO.3480sec.scaled.stack.fits'

print(f"#\t{len(science_images)} Science Images found")
for ss, sci_image in enumerate(science_images):
    print(f"[{ss}] {os.path.basename(sci_image)}")
print(f"--> Reference Image: {os.path.basename(reference_image)}")

: 

## Trim Configuration

In [ ]:
number_x_split = 4
number_y_split = 4

x_indx_ranges = np.arange(number_x_split)
y_indx_ranges = np.arange(number_y_split)

number_trim_images = number_x_split*number_y_split

print(f"Image Split: {number_x_split} x {number_y_split} = {number_trim_images} Images")

: 

## Science Image
- data array
- header
- WCS

In [ ]:
nn = 0
sci_image = science_images[nn]
# HDU
sci_hdu = fits.open(sci_image)[0]
# Load the image and the WCS
sci_wcs = WCS(sci_hdu.header)
# Put the cutout image in the FITS HDU
sci_data = sci_hdu.data
# Update the FITS header with the cutout WCS
sci_hdr = sci_hdu.header

: 

In [ ]:
xshape, yshape = sci_data.shape

x_trim_step = int(xshape/number_x_split)
y_trim_step = int(yshape/number_y_split)

print(f"Full image: {xshape}x{yshape}")
print(f"-> {x_trim_step:g}x{y_trim_step:g}")

: 

### Trim the science image

In [ ]:
#	Trim Number
tt = 0
xx, yy = 0, 0

st = time.time()
for xx in x_indx_ranges:
	for yy in y_indx_ranges:
		print(f"[{tt}] Trim --> Sector:({xx},{yy})")
		# trim_sci_image = sci_image.replace('fits', f"trim_{tt:0>2}.fits")
		trim_sci_image = rename_convention(sci_image).replace('fits', f"trim_{xx}x{yy}.fits")
		trim_sci_data = sci_data[yy*y_trim_step:(yy+1)*y_trim_step, xx*x_trim_step:(xx+1)*x_trim_step]
		trim_sci_hdr = sci_hdr.copy()
		trim_sci_hdr['TRIM'] = int(tt)
		trim_sci_wcs = sci_wcs[yy*y_trim_step:(yy+1)*y_trim_step, xx*x_trim_step:(xx+1)*x_trim_step]

		trim_sci_hdu = fits.PrimaryHDU(data=trim_sci_data, header=trim_sci_hdr)
		trim_sci_hdu.header.update(trim_sci_wcs.to_header())

		# trim_sci_hdu.writeto(trim_sci_image, overwrite=True)
		if not os.path.exists(trim_sci_image):
			trim_sci_hdu.writeto(trim_sci_image, overwrite=True)
		else:
			print("Already exists")
		tt += 1
delt = time.time() - st
print(f"Done ({delt:g} sec)")

: 

## Reference Image

In [ ]:
ref_image = reference_image
# HDU
ref_hdu = fits.open(ref_image)[0]
# Load the image and the WCS
ref_wcs = WCS(ref_hdu.header)
# Put the cutout image in the FITS HDU
ref_data = ref_hdu.data
# Update the FITS header with the cutout WCS
ref_hdr = ref_hdu.header

: 

In [ ]:
#	Trim Number
tt = 0
xx, yy = 0, 0

st = time.time()
for xx in x_indx_ranges:
	for yy in y_indx_ranges:
		print(f"[{tt}] Trim --> Sector:({xx},{yy})")
		# trim_ref_image = ref_image.replace('fits', f"trim_{tt:0>2}.fits")
		trim_ref_image = rename_convention(ref_image).replace('fits', f"trim_{xx}x{yy}.fits").replace("Calib", "REF")
		trim_ref_data = ref_data[yy*y_trim_step:(yy+1)*y_trim_step, xx*x_trim_step:(xx+1)*x_trim_step]
		trim_ref_hdr = ref_hdr.copy()
		trim_ref_hdr['TRIM'] = int(tt)
		trim_ref_wcs = ref_wcs[yy*y_trim_step:(yy+1)*y_trim_step, xx*x_trim_step:(xx+1)*x_trim_step]

		trim_ref_hdu = fits.PrimaryHDU(data=trim_ref_data, header=trim_ref_hdr)
		trim_ref_hdu.header.update(trim_ref_wcs.to_header())

		trim_ref_hdu.writeto(trim_ref_image, overwrite=True)
		# if not os.path.exists(trim_ref_image):
		# 	trim_ref_hdu.writeto(trim_ref_image, overwrite=True)
		# else:
		# 	print("Already exists")
		tt += 1
delt = time.time() - st
print(f"Done ({delt:g} sec)")

: 

# Subtraction Routine

In [ ]:
# trim_science_images = sorted(glob.glob(f"{path_science_image}/TOO*trim_*.fits"))
# trim_reference_images = sorted(glob.glob(f"{path_science_image}/REF*trim_*.fits"))
trim_science_images = sorted(glob.glob(f"{path_science_image}/Calib*trim_*.fits"))
trim_reference_images = sorted(glob.glob(f"{path_science_image}/REF*trim_*.fits"))

print(f"Number of Trimmed Science images  : {len(trim_science_images)}")
print(f"Number of Trimmed Reference images: {len(trim_reference_images)}")

if len(trim_science_images) == len(trim_reference_images):
    print(f"--> Same Number of Sci & Ref Images")
elif len(trim_science_images) > len(trim_reference_images):
    print(f"--> N_sci > N_ref")
else:
    print(f"--> N_sci < N_ref")

: 

In [ ]:
numbers = np.arange(number_trim_images)
for number in numbers:
# for number in np.arange(number_trim_images)[0:1]:
    trim_sci_image = trim_science_images[number]
    trim_ref_image = trim_reference_images[number]
    #   Align
    #----------------------------------------------------------------
    #   If alignment is needed,
    #----------------------------------------------------------------
    # align_ref_image = trim_ref_image.replace("REF", "wrREF")
    # if os.path.exists(align_ref_image):
    #     os.system(f"rm {align_ref_image}")
    # wcsremap(inim=trim_sci_image, refim=trim_ref_image, outim=align_ref_image, path_com='/data3/wcsremap/wcsremap-1.0.1/wcsremap')
    #----------------------------------------------------------------
    #   If alignment is not needed,
    #----------------------------------------------------------------
    align_ref_image = trim_ref_image

    #   Subtraction
    # trim_subt_image = trim_sci_image.replace("TOO", "hdTOO")
    trim_subt_image = trim_sci_image.replace("Calib", "hdCalib")
    # convolved_ref_image = align_ref_image.replace("wrREF", "hcwrREF")
    convolved_ref_image = align_ref_image.replace("REF", "hcREF")
    # if os.path.exists(trim_subt_image):
    #     os.system(f"rm {trim_subt_image}")
    # if os.path.exists(convolved_ref_image):
    #     os.system(f"rm {convolved_ref_image}")
    if (os.path.exists(trim_subt_image) == False) & (os.path.exists(convolved_ref_image) == False):
        hotpants(inim=trim_sci_image, refim=align_ref_image, outim=trim_subt_image, convim=convolved_ref_image)

: 

# Transient Search

In [ ]:
path_config = '/home/paek/config'
conv, nnw, param, sex = 'transient.kmtnet.conv', 'transient.kmtnet.nnw', 'transient.kmtnet.param', 'transient.kmtnet.sex'
conf_sex = '{}/{}'.format(path_config, sex)
conf_param = '{}/{}'.format(path_config, param)
conf_nnw = '{}/{}'.format(path_config, nnw)
conf_conv = '{}/{}'.format(path_config, conv)

: 

In [ ]:
# number = 3
for number in numbers:
	#	Input Images
	trim_sci_image = trim_science_images[number]
	trim_ref_image = trim_reference_images[number]
	print(f"[{number}] {os.path.basename(trim_sci_image)}")

	#	Middle-step images
	align_ref_image = trim_ref_image
	trim_subt_image = trim_sci_image.replace("Calib", "hdCalib")
	convolved_ref_image = align_ref_image.replace("REF", "hcREF")

	#------------------------------------------------------------
	#	Invert the images
	#------------------------------------------------------------
	inverse_trim_subt_image = trim_subt_image.replace("hd", "invhd")
	invert_image(inim=trim_subt_image, outim=inverse_trim_subt_image)
	inverse_convolved_ref_image = convolved_ref_image.replace("hc", "invhc")
	invert_image(inim=convolved_ref_image, outim=inverse_convolved_ref_image)
	#------------------------------------------------------------
	#	Photometry
	#------------------------------------------------------------
	#	Subtraction
	os.system(sexcom(trim_subt_image, conf_sex, conf_param, conf_conv, conf_nnw))
	#	Convolved Reference
	os.system(sexcom(convolved_ref_image, conf_sex, conf_param, conf_conv, conf_nnw))
	#	Inverted Subtraction
	os.system(sexcom(inverse_trim_subt_image, conf_sex, conf_param, conf_conv, conf_nnw))
	#	Inverted Reference
	os.system(sexcom(inverse_convolved_ref_image, conf_sex, conf_param, conf_conv, conf_nnw))
	#------------------------------------------------------------
	#	Output Catalog
	#------------------------------------------------------------
	#	Subtraction
	trim_subt_cat = trim_subt_image.replace("fits", "cat")
	#	Convolved Reference
	convolved_ref_cat = convolved_ref_image.replace("fits", "cat")
	#	Inverted Subtraction
	inverse_subt_cat = inverse_trim_subt_image.replace("fits", "cat")
	#	Inverted Reference
	inverse_convolved_ref_cat = inverse_convolved_ref_image.replace("fits", "cat")
	#------------------------------------------------------------
	#	Read Catalog
	#------------------------------------------------------------
	reftbl = ascii.read(convolved_ref_cat)
	subtbl = ascii.read(trim_subt_cat)
	invsubtbl = ascii.read(inverse_subt_cat)
	invreftbl = ascii.read(inverse_convolved_ref_cat)
	print(f"# Number of sources: {len(subtbl)}")
	#------------------------------------------------------------
	#	Mark the Flags
	#------------------------------------------------------------
	xcent = int(x_trim_step/2)
	ycent = int(y_trim_step/2)
	frac = 0.99
	sep = 1.0
	fovval = 1.0*60 # [arcmin]
	#------------------------------------------------------------
	subtbl = ascii.read(trim_subt_cat)
	subtbl['inim'] = trim_sci_image
	subtbl['hcim'] = convolved_ref_image
	subtbl['hdim'] = trim_subt_image
	subtbl['snr'] = 1/subtbl['MAGERR_AUTO']
	#	Seeing
	subtbl.meta['SEEING'] = np.median(subtbl['FWHM_WORLD']*3600)
	subtbl['ratio_seeing'] = subtbl['FWHM_WORLD']*3600/subtbl.meta['SEEING']
	#	Ellipticity
	subtbl.meta['ELLIPTICITY'] = np.median(subtbl['ELLIPTICITY'])
	subtbl['ratio_ellipticity'] = subtbl['ELLIPTICITY']/subtbl.meta['ELLIPTICITY']
	#	Elongation
	subtbl.meta['ELONGATION'] = np.median(subtbl['ELONGATION'])
	subtbl['ratio_elongation'] = subtbl['ELONGATION']/subtbl.meta['ELONGATION']
	#	Magnitude Calibration (ZP=30)
	subtbl['mag_auto'] = subtbl['MAG_AUTO']+30

	w = WCS(trim_subt_image)
	#	Positional information
	c_cent = w.pixel_to_world(xcent, ycent)
	c_sub = SkyCoord(subtbl['ALPHA_J2000'], subtbl['DELTA_J2000'], unit=u.deg)

	flagnumbers = np.arange(14)
	#	Generate flag columns
	for num in flagnumbers:
		subtbl[f'flag_{num}'] = False
	epoch = Time(fits.getheader(trim_sci_image)['DATE-OBS'], format='isot')
	#	
	if "CTIO" in trim_sci_image:
		location="807"
	elif "SSO" in trim_sci_image:
		location="Q60"
	elif "SAAO" in trim_sci_image:
		location="M22"
	#------------------------------------------------------------
	#	flag 0
	#------------------------------------------------------------
	#	Skybot query
	try:
		sbtbl = Skybot.cone_search(c_cent, fovval*u.arcmin, epoch, location=location)
		c_sb = SkyCoord(sbtbl['RA'], sbtbl['DEC'])
		sbtbl['sep'] = c_cent.separation(c_sb).to(u.arcmin)
		#	Skybot matching
		indx_sb, sep_sb, _ = c_sub.match_to_catalog_sky(c_sb)
		subtbl['flag_0'][(sep_sb.arcsec<5)] = True
	except RuntimeError:
		print(f'No solar system object was found in the requested FOV ({fovval} arcmin)')
	#------------------------------------------------------------
	#	flag 1
	#------------------------------------------------------------
	if len(invsubtbl)>0:
		#	Coordinate
		c_invhd = SkyCoord(invsubtbl['ALPHA_J2000'], invsubtbl['DELTA_J2000'], unit=u.deg)
		#	Matching with inverted images
		indx_invhd, sep_invhd, _ = c_sub.match_to_catalog_sky(c_invhd)
		subtbl['flag_1'][(sep_invhd.arcsec<subtbl.meta['SEEING'])] = True
	else:
		print('Inverted subtraction image has no source. ==> pass flag1')
		pass
	#------------------------------------------------------------
	#	flag 2
	#------------------------------------------------------------
	if len(invreftbl)>0:
		#	Coordinate
		c_invhc = SkyCoord(invreftbl['ALPHA_J2000'], invreftbl['DELTA_J2000'], unit=u.deg)
		#	Matching with inverted images
		indx_invhc, sep_invhc, _ = c_sub.match_to_catalog_sky(c_invhc)
		subtbl['flag_2'][(sep_invhc.arcsec<subtbl.meta['SEEING'])] = True
	else:
		print('Inverted reference image has no source. ==> pass flag2')
		pass
	#------------------------------------------------------------
	#	SEtractor criterion
	#------------------------------------------------------------
	#	flag 3
	#------------------------------------------------------------
	#	Sources @edge
	# frac = 0.99
	subtbl['flag_3'][
		((subtbl['X_IMAGE']<xcent-xcent*frac) |
		(subtbl['X_IMAGE']>xcent+xcent*frac) |
		(subtbl['Y_IMAGE']<ycent-ycent*frac) |
		(subtbl['Y_IMAGE']>ycent+ycent*frac))
		] = True
	#------------------------------------------------------------
	#	flag 4 --> skip
	#------------------------------------------------------------
	#	More than 5 sigma signal
	# subtbl['flag_4'][(subtbl['mag_aper']>hdr['ul5_1'])] = True
	#	Empirical criterion
	#------------------------------------------------------------
	#	flag 5
	#------------------------------------------------------------
	subtbl['flag_5'][(subtbl['ratio_ellipticity'] > 5)] = True
	#------------------------------------------------------------
	#	flag 6
	#------------------------------------------------------------
	subtbl['flag_6'][(subtbl['FLAGS'] > 4.0)] = True
	#------------------------------------------------------------
	#	flag 7
	#------------------------------------------------------------
	subtbl['flag_7'][
		# (subtbl['FWHM_WORLD']*3600>subtbl.meta['SEEING']*3.0) |
		# (subtbl['FWHM_WORLD']*3600<subtbl.meta['SEEING']*0.5)
		# (subtbl['FWHM_WORLD']*3600<subtbl.meta['SEEING']*0.8) |
		# (subtbl['FWHM_WORLD']*3600>subtbl.meta['SEEING']*2.0) 
		(subtbl['ratio_seeing']<0.8) |
		(subtbl['ratio_seeing']>2.0)
		] = True
	#------------------------------------------------------------
	#	flag 8
	#------------------------------------------------------------
	subtbl['flag_8'][
		(subtbl['BACKGROUND']<-50) |
		(subtbl['BACKGROUND']>+50)
		] = True
	#------------------------------------------------------------
	#	flag 9 --> skip
	#------------------------------------------------------------
	subtbl['flag_9'][
		(subtbl['snr']<5)
	] = True
	#------------------------------------------------------------
	#	flag 10+11
	#------------------------------------------------------------
	data = fits.getdata(trim_subt_image)
	peeing = subtbl.meta['SEEING']/0.4
	skyval = np.median(subtbl['BACKGROUND'])
	skysig = np.std(subtbl['BACKGROUND'])

	nbadlist = []
	ratiobadlist = []
	nnulllist = []

	subtbl['n_bad'] = 0
	subtbl['ratio_bad'] = 0.0
	subtbl['n_null'] = 0
	#	Fraction
	f = 0.3
	for i, (tx, ty, bkg) in enumerate(zip(subtbl['X_IMAGE'], subtbl['Y_IMAGE'], subtbl['BACKGROUND'])):

		# tx, ty = subtbl['X_IMAGE'][i], subtbl['Y_IMAGE'][i]
		# bkg = subtbl['BACKGROUND'][i]
		#	Snapshot
		tsize = peeing
		y0, y1 = int(ty-tsize), int(ty+tsize)
		x0, x1 = int(tx-tsize), int(tx+tsize)
		cdata = data[y0:y1, x0:x1]
		# plt.close()
		# plt.imshow(cdata)
		crt = bkg - skysig*5
		cutline = cdata.size*f
		nbad = len(cdata[cdata<crt])
		try:
			ratiobad = nbad/cdata.size
		except:
			ratiobad = -99.0
		nnull = len(np.where(cdata == 1e-30)[0])
		#	Dipole
		if nbad > cutline:
			subtbl['flag_10'][i] = True
		#	HOTPANTS Null value
		if nnull != 0:
			subtbl['flag_11'][i] = True

		subtbl['n_bad'][i] = nbad
		subtbl['ratio_bad'][i] = ratiobad
		subtbl['n_null'][i] = nnull

	#------------------------------------------------------------
	#	flag 12
	#------------------------------------------------------------
	# x, y = fits.getdata(hcim).shape
	# w_ref = WCS(hcim)
	# xim, yim = w_ref.world_to_pixel(c_sub)
	# indx_nosci = np.where(
	# 	(xim < 0) | (xim > x) | (yim < 0) | (yim > y)
	# )
	# subtbl['flag_12'][indx_nosci] = True
	x, y = fits.getdata(convolved_ref_image).shape
	w_ref = WCS(convolved_ref_image)
	xim, yim = w_ref.world_to_pixel(c_sub)
	subtbl['x_refim'] = xim
	subtbl['y_refim'] = yim
	indx_nosci = np.where(
		(xim < 0) | (xim > x) | (yim < 0) | (yim > y)
	)
	subtbl['flag_12'][indx_nosci] = True
	#------------------------------------------------------------
	#	flag 13
	#------------------------------------------------------------
	subtbl['n_saturation_line'] = int(0)
	subtbl['ratio_saturation_line'] = 0.0
	y_to_check = np.array([num for num in np.arange(5, 140+10, 10) if num !=75])

	for nn, (tx, ty) in enumerate(zip(subtbl['X_IMAGE'], subtbl['Y_IMAGE'])):
		tsize = int((60/0.4)/2)
		y0, y1 = int(ty-tsize), int(ty+tsize)
		x0, x1 = int(tx-tsize), int(tx+tsize)
		_data = data[y0:y1, x0:x1]

		ct = 0
		xshp, yshp = _data.shape
		if (xshp > 140) & (yshp > 140) & (xshp == yshp):
			for ny, yy in enumerate(y_to_check):
				center_mean_val = np.mean(_data[yy, int(xshp/2)-3:int(xshp/2)+3])
				local_bkg = np.median(_data[yy, :])
				ct += 1 if center_mean_val > local_bkg else 0
			# print(f"N={nn}: {ct}")
			subtbl['n_saturation_line'][nn] = ct
			subtbl['ratio_saturation_line'][nn] = ct/len(y_to_check)
	subtbl['flag_13'][subtbl['ratio_saturation_line'] > 0.7] = True
	#------------------------------------------------------------
	#	Final flag
	#------------------------------------------------------------
	subtbl['flag'] = False
	flag = subtbl['flag']
	n_all = len(subtbl)
	for n in flagnumbers:
		tmptbl = subtbl[subtbl[f'flag_{n}']==True] 
		print(f'flag=={n} : {len(tmptbl)} {int(100*len(tmptbl)/n_all)}%')
		flag = flag + subtbl[f'flag_{n}']
	subtbl['flag'] = flag
	indx_sb = np.where(subtbl['flag_0']==True)
	subtbl['flag'][indx_sb] = False
	#	Transient Catalog
	trtbl = subtbl[subtbl['flag']==False]
	transient_cat = trim_subt_cat.replace('cat', 'transient.cat')
	subtbl.write(transient_cat, format='ascii.tab', overwrite=True)
	print('-'*60)
	print(f'Filtered sources\t: {len(trtbl)} ({100*len(trtbl)/len(subtbl):1.3f})%')
	#------------------------------------------------------------
	#	Log
	#------------------------------------------------------------
	logname = transient_cat.replace("cat", "summary.txt")
	f = open(logname, 'w')
	for n in flagnumbers:
		tmptbl = subtbl[subtbl[f'flag_{n}']==True] 
		line = f'flag=={n}: {len(tmptbl)} {int(100*len(tmptbl)/n_all)}%\n'
		f.write(line)
	line = f'Filtered sources\t: {len(trtbl)} ({100*len(trtbl)/len(subtbl):1.3f})%'
	f.write(line)
	f.close()


	import multiprocessing
	from itertools import repeat

	cutsize = 1.0
	# ncore = 4
	ncore = 1
	#------------------------------------------------------------
	#	Snapshot maker
	#------------------------------------------------------------
	print(f"#\tSnapshot maker ({len(trtbl)})")
	# cutsize=0.5
	if len(trtbl) > 0:
		#	Single Thread
		for i in range(len(trtbl)):
			generate_snapshot(trtbl, i, cutsize)

		#	Multi Thread
		# if __name__ == '__main__':
		# 	with multiprocessing.Pool(processes=ncore) as pool:
		# 		results = pool.starmap(generate_snapshot, zip(repeat(trtbl), np.arange(len(trtbl)), repeat(cutsize)))
	else:
		print('No transient candidates.')
print("All Done")

: 